# Kolymbia Bay energy system model

Modelling two variations of rooftop PV panels (IBC and Thin film) on buildings 1, 2, 4 and, 5 as well as BIPV on balconies (buildings 1-3) and the south facade of building 1. 
Buildings 1,2,4 and, 5 have roof area available for rooftop PV at areas of 120, 110, 84 and 140 (sqm) respectively. 
Balcony area available on buildings 1, 2 and, 3 are 36, 10 and 36 (sqm) respectively.
Facade area available for BIPV on building 1 = 17sqm. 


Modelling each building as a different location to account for installation costs which include DC cables to connect each building to the energy metre. 
Each building labelled Building1, Building2 ....
and the energy metre labelled as KolymbiaBay

7 model runs:
1. Utility only 
2. 25 degrees tilt, no limit to CO2 emissions (optimal cost), full energy demand
3. 25 degrees tilt, max limit to CO2 emissions, full energy demand
4. 8 degrees tilt, no limit to CO2 emissions (optimal cost), full energy demand
5. 8 degrees tilt, max limit to CO2 emissions, full energy demand
6. 8 degrees tilt, no limit to CO2 emissions (optimal cost), reduced energy demand
7. 8 degrees tilt, max limit to CO2 emissions, reduced energy demand

max limit to CO2 emissions found when CO2 reduction target results in an 'infeasible solution' meaning the energy system is not sustainable without greater inputs from the utility provider.

The listed runs will analyse:

- the preferred PV tech 
1. high efficency, high cost 
2. low efficiency, low cost
3. BIPV

- Optimum tilt angle. 
The optimum angle varies throughout the year. Will the PV system be more effective with the optimum tilt required for max generation in July and August when demand is high (8 degrees) or is it better to have an average tilt that is optimal for annual generation (25 degrees).

- The effect of energy saving strategies
Looking at hotel specific data ~ 50% of energy demand during summer accounts for cooling. Can run same model with 80% energy demand during the months April-October to mimic reduced energy consumption. 

This is carried out at hourly timesteps

Inverter = Power Electronics: FS1003CU

Comparing the CO2 emissions and cost associated with each combination. 

## Sources
1. PV IBC, Roof 1
2. PV IBC, Roof 2
3. PV IBC, Roof 4
4. PV IBC, Roof 5


5. Thin film, Roof 1
6. Thin film, Roof 2
7. Thin film, Roof 4
8. Thin film, Roof 5


9. BIPV, Balcony 1
10. BIPV, Balcony 2
11. BIPV, Balcony 3
12. BIPV, South facade



13. Utility provider (electricity)



# Sinks
1. Hotel Electricity Demand 
2. CO2 to environment 
3. Utility provider (for surplus generation)

# Conversion 
1. Inverter 
2. Virtual CO2 conversion (energy from prover to CO2 emissions)

# Storage 
No storage - assuming net metering system only

# Transmission

Cables between buildings

Set hotel as a location - 1 demand for 'kolymbia Bay' 0 for the rest (5 buildings).
Hotel location doesnt have any potential to produce energy from PV
Transmission to transfer from PV generation on buildings to demand in energy metre 


# Import packages

In [ ]:
import FINE as fn
from getData import getData

import pandas as pd
import os
import openpyxl

cwd = os.getcwd()
data = getData()


# Input parameters

In [ ]:
# define locations 
locations = {'Building1', 'Building2', 'Building3', 'Building4', 'Building5', 'KolymbiaBay'} #need hotel as location??
commodityUnitDict = {"powerAC": r"kW$_{el}$", "powerDC": r"kW$_{el}$", "CO2": r"kg$_{CO_2}$/h", "utility":r"kW$_{el}$"}
commodities = {"powerAC", "powerDC", "CO2", "utility" }
numberOfTimeSteps = 8760
hoursPerTimeStep = 1

# Code
esM = fn.EnergySystemModel(locations=locations, commodities=commodities,
    numberOfTimeSteps=numberOfTimeSteps, commodityUnitsDict=commodityUnitDict,
    hoursPerTimeStep=hoursPerTimeStep, verboseLogLevel=0)

# Create energy system model

In [ ]:
esM = fn.EnergySystemModel(
    locations=locations,
    commodities=commodities,
    numberOfTimeSteps=numberOfTimeSteps,
    commodityUnitsDict=commodityUnitDict,
    hoursPerTimeStep=hoursPerTimeStep,
    costUnit="Euro",
    lengthUnit="km",
    verboseLogLevel=0,
)

# Source: Interdigitated Back Contact (IBC) PV

Add data

Change operation_25 to operation_8 in variations 4,5,6,7

For 'utility only' comment out all PV sources 

In [ ]:
IBC_operationRateMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/IBC.xlsx'),  
                                     sheet_name ='operation_25',header = 0, index_col = 0, engine='openpyxl')
IBC_capacityMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/IBC.xlsx'),  
                                sheet_name ='capacities', index_col=0, engine='openpyxl',squeeze=True)

Add source to model

In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='PV_IBC', 
                  commodity='powerDC', 
                  hasCapacityVariable=True,
                  operationRateMax=IBC_operationRateMax, 
                  capacityMax=IBC_capacityMax,
                  investIfBuilt=700, #labour cost 2 days
                  investPerCapacity=812.3, #total cost for given area divided by capacity for that area
                  opexPerCapacity=812.3*0.02, #maintainence
                  interestRate=0.08, #assume 8 percent
                  economicLifetime=25,
                  hasIsBuiltBinaryVariable=True,
                  bigM=50,
                  sharedPotentialID='rooftop')) #prevents IBC and thinfilm at same time



# Source: PV thin film 

Add data

Change operation_25 to operation_8 in variations 4,5,6,7

For 'utility only' comment out all PV sources 

In [ ]:
thinfilm_operationRateMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/thinfilm.xlsx'),  
                                     sheet_name ='operation_25',header = 0, index_col = 0, engine='openpyxl')
thinfilm_capacityMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/thinfilm.xlsx'),  
                                sheet_name ='capacities', index_col=0, engine='openpyxl',squeeze=True)

In [ ]:
#less maintainance and cheaper to install, shorter economiclifetime
esM.add(fn.Source(esM=esM, 
                  name='PV_thinfilm', 
                  commodity='powerDC', 
                  hasCapacityVariable=True,
                  operationRateMax=thinfilm_operationRateMax, 
                  capacityMax=thinfilm_capacityMax,
                  investIfBuilt=700, #labour cost 2 days 
                  investPerCapacity=1053, #total cost for given area divided by capacity for that area
                  opexPerCapacity=1053*0.01, #maintainence
                  interestRate=0.08, #assume 8 percent
                  economicLifetime=20,
                  hasIsBuiltBinaryVariable=True,
                  bigM=50,
                  sharedPotentialID='rooftop')) #shared potential to stop IBC and thinfilm on same area.
                 

# Source: BIPV balcony 

Add data

For 'utility only' comment out all PV sources 

In [ ]:
BALC_operationRateMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/PV_BALC.xlsx'),  
                                     sheet_name ='BALC_operation_hourly_group',header = 0, index_col = 0, engine='openpyxl')
BALC_capacityMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/PV_BALC.xlsx'),  
                                sheet_name ='BALC_capacities', index_col=0, engine='openpyxl',squeeze=True)

Add source

In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='PV_BALC', 
                  commodity='powerDC', 
                  hasCapacityVariable=True,
                  operationRateMax=BALC_operationRateMax, 
                  capacityMax=BALC_capacityMax,
                  investIfBuilt=700, #labour cost 2 days
                  investPerCapacity=407.5, #total cost for given area divided by capacity for that area
                  opexPerCapacity=407.5*0.03, #maintainence greater
                  interestRate=0.08,
                  hasIsBuiltBinaryVariable=True,
                  bigM=50,
                  economicLifetime=35,))
               

#ONYX

# Source: BIPV south facade

For 'utility only' comment out all PV sources 

In [ ]:
SOUTH_operationRateMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/PV_SOUTH.xlsx'),  
                                     sheet_name ='SOUTH_operation_hourly_group',header = 0, index_col = 0, engine='openpyxl')
SOUTH_capacityMax = pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/PVtech/PV_SOUTH.xlsx'),  
                                sheet_name ='SOUTH_capacities', index_col=0, engine='openpyxl',squeeze=True)


In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='PV_SOUTH', 
                  commodity='powerDC', 
                  hasCapacityVariable=True,
                  operationRateMax=SOUTH_operationRateMax, 
                  capacityMax=SOUTH_capacityMax,
                  investIfBuilt=700, #labour cost 2 days
                  investPerCapacity=696.7, 
                  opexPerCapacity=696.7*0.02,
                  hasIsBuiltBinaryVariable=True,
                  bigM=50,
                  interestRate=0.08,
                  economicLifetime=25))

# Source: Electricity provider

In [ ]:
# import locationalEligibility Dataframe
CO2_locs= pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/LocationalEligability.xlsx'),  
                                sheet_name ='Locs', index_col=0, engine='openpyxl',squeeze=True)

In [ ]:
esM.add(fn.Source(esM=esM, 
                  name='Utility', 
                  commodity='utility', 
                  hasCapacityVariable=False,
                  commodityCost=0.0779, # from energy bills (euro/kWh)
                  locationalEligibility=CO2_locs,
                )) 


# Conversion: Virtual Conversion CO2

In [ ]:
esM.add(
            fn.Conversion(
                esM=esM,
                name="CO2Emissions",
                physicalUnit=r"kW$_{el}$",
                commodityConversionFactors={"utility": -1, "powerAC": 1, "CO2":0.42},
                locationalEligibility=CO2_locs,
                
            )
        ) 
#  CO2 component - 0.42kg/kWh

# Conerverion: Inverter

In [ ]:

esM.add(
            fn.Conversion(
                esM=esM,
                name="Inverter",
                physicalUnit=r"kW$_{el}$",
                commodityConversionFactors={"powerAC": 0.97, "powerDC": -1},
                locationalEligibility=CO2_locs,
            )
        )

# Transmisson

add data for locationaleligibility and cost of cabling between buildings and energy metre. Assumed locations joined as follows:
1 joins to 2
2 joins to 3
3 joins to 4
4 joins to metre
5 joins to metre

In [ ]:
Trans_locs= pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/Transmission.xlsx'),  
                                sheet_name ='LocationalE', index_col=0, engine='openpyxl',squeeze=True)
Trans_cost= pd.read_excel(os.path.join('/Users/zaramulholland/Desktop/Data/Transmission.xlsx'),  
                                sheet_name ='Cost', index_col=0, engine='openpyxl',squeeze=True)

In [ ]:
esM.add(fn.Transmission(esM=esM, 
                        name='AC cables', 
                        commodity='powerAC',
                        hasCapacityVariable=True,
                        hasIsBuiltBinaryVariable=True,
                        locationalEligibility=Trans_locs,
                        investIfBuilt=Trans_cost,
                        bigM=50))

In [ ]:
esM.add(fn.Transmission(esM=esM, 
                        name='DC cables', 
                        commodity='powerDC',
                        hasCapacityVariable=True,
                        hasIsBuiltBinaryVariable=True,
                        locationalEligibility=Trans_locs,
                        investIfBuilt=Trans_cost,
                        bigM=100))
                        

# Sink: Electricity demand 

For reduced demand - read excel sheet ReducedDemand2

In [ ]:
# Electricity demand data
    operationRateFix = pd.read_excel(os.path.join("/Users/zaramulholland/Desktop/Data/Demand/Hotel_EnergyDemand.xlsx"), 
                                     sheet_name='HotelDemand', header=0, index_col = 0,  engine='openpyxl')




In [ ]:
# Add sink
esM.add(
    fn.Sink(
        esM=esM,
        name="Electricity demand",
        commodity="powerAC",
        hasCapacityVariable=False,
        operationRateFix=operationRateFix,
    )
)

# Sink: Add utility provider as a sink (surplus from PV generation)

In [ ]:
esM.add(
    fn.Sink(
        esM=esM,
        name="Surplus to Utility",
        commodity="powerAC",
        hasCapacityVariable=False,
        locationalEligibility=CO2_locs  
    )
)



# Sink: CO2 to environment

In [ ]:
CO2_reductionTarget = 1

In [ ]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False,
                locationalEligibility=CO2_locs,
                commodityLimitID='CO2 limit',
                
                yearlyLimit=161599*(CO2_reductionTarget))) # 161599 = CO2 emissions when all from utility
               


In [ ]:
# Optimise (100 for increased accuracy) 

esM.cluster(numberOfTypicalPeriods=10)

In [ ]:
esM.optimize(timeSeriesAggregation=True, solver="glpk")

In [ ]:
srcSnkSummary = esM.getOptimizationSummary("SourceSinkModel", outputLevel=1)
display(esM.getOptimizationSummary("SourceSinkModel", outputLevel=2))

In [ ]:
srcSnkSummary

In [ ]:

#fn.writeOptimizationOutputToExcel(
 #   esM,
  #  outputFileName="/Users/zaramulholland/Desktop/Results/25degrees_noCO2limit",
   # optSumOutputLevel=0,
    #optValOutputLevel=0,)

In [ ]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

In [ ]:
#fig, ax = fn.plotOperationColorMap(esM, "PV_IBC", "Building1")

In [ ]:
#fig, ax = fn.plotOperationColorMap(esM, "PV_IBC", "Building2")

In [ ]:
#fig, ax = fn.plotOperationColorMap(esM, "PV_BALC", "Building1")

In [ ]:
#fig, ax = fn.plotOperationColorMap(esM, "PV_BALC", "Building3")

although daily operation period is short, still cost effienct as allows an additional energy input at what the model deems a realistic investiment cost.

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, "CO2 to enviroment", "KolymbiaBay", vmax=50)

In [ ]:
fig, ax = fn.plotOperationColorMap(esM, 'Electricity demand', 'KolymbiaBay')

In [ ]:
fig, ax = fn.plotOperation(esM, 'Utility', 'KolymbiaBay')

Electricity taken from utility provider correlates with CO2 emissions associated with the hotel. This is the only thing related to CO2 in the model. Aim to minimise this and minimise cost. This will change significanty if I great a typical daily profile and apply to demand.